# Evidence for water worlds around FGK stars
## Water-world means following Zeng et al. (2019)

**Author: [Hannu Parviainen](mailto:hpparvi@gmail.com)** </br>
**Last edited: 25. October 2023**

This notebook repeats the analysis of the 01a_stmp_zeng19 notebook, but for the TEPCat catalogue of small planets around FGK stars. Read the previous notebook for details.

In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
from dynesty import NestedSampler
from uncertainties import ufloat

from spright import RMEstimator
from spright.io import read_tepcat
from spright.model import lnlikelihood_sample

%run common.py

In [5]:
names, radii, masses = read_tepcat('../../spright/data/TEPCat_FGK_20230522.csv')
rme = RMEstimator(nsamples=200, names=names, radii=radii, masses=masses)
lpf = rme.lpf
rdm = lpf.rdm

In [ ]:
def transform_h0(u):
    return transform_base(u, 0.0, 0.001, water_model='z19')

def transform_h1(u):
    return transform_base(u, 0.15, 0.5, water_model='z19')

def transform_h2(u):
    return transform_base(u, 0.5, 1.0, water_model='z19')

In [6]:
def lnlikelihood(pv):
    if is_puffy_population_ok(pv, lpf.rdm):
        lnl = lnlikelihood_sample(pv, lpf.density_samples, lpf.radius_samples,
                                  rdm._rr0, rdm._rdr, rdm._rx0, rdm._rdx, rdm.drocky, 
                                  rdm._wr0, rdm._wdr, rdm._wx0, rdm._wdx, rdm.dwater)
        return lnl if isfinite(lnl) else -1e6
    else:
        return -1e6

In [7]:
s0 = NestedSampler(lnlikelihood, transform_h0, 11, nlive=1000)
s1 = NestedSampler(lnlikelihood, transform_h1, 11, nlive=1000)
s2 = NestedSampler(lnlikelihood, transform_h2, 11, nlive=1000)

In [8]:
s0.run_nested()

12302it [03:37, 56.45it/s, +1000 | bound: 59 | nc: 1 | ncall: 308950 | eff(%):  4.320 | loglstar:   -inf < -264.907 <    inf | logz: -277.493 +/-  0.130 | dlogz:  0.001 >  1.009]


In [9]:
s1.run_nested()

14617it [05:14, 46.45it/s, +1000 | bound: 75 | nc: 1 | ncall: 381645 | eff(%):  4.103 | loglstar:   -inf < -261.224 <    inf | logz: -276.263 +/-  0.160 | dlogz:  0.001 >  1.009]


In [10]:
s2.run_nested()

14621it [05:21, 45.43it/s, +1000 | bound: 74 | nc: 1 | ncall: 378623 | eff(%):  4.137 | loglstar:   -inf < -261.172 <    inf | logz: -276.179 +/-  0.151 | dlogz:  0.001 >  1.009]


In [13]:
z0 = ufloat(s0.results.logz[-1], s0.results.logzerr[-1])
z1 = ufloat(s1.results.logz[-1], s1.results.logzerr[-1])
z2 = ufloat(s2.results.logz[-1], s2.results.logzerr[-1])

In [14]:
print(2*(z1 - z0), 2*(z2 - z0), 2*(z2 - z1))

2.5+/-0.7 2.6+/-0.7 0.2+/-0.7


---

<center>
©2023 Hannu Parviainen
</center>